## eye-tracking experiments

In [124]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

variables

In [2]:
features = [
    'Category Right',
    'Category Left',
    'Point of Regard Right X [px]',
    'Point of Regard Right Y [px]',
    'Point of Regard Left X [px]',
    'Point of Regard Left Y [px]'
]
categorical = {
    'Category Right': ['Blink', 'Saccade', 'Fixation'],
    'Category Left': ['Blink', 'Saccade', 'Fixation']
}

load data

In [116]:
dfs = []
for file in os.listdir('split/'):
    dfs.append(pd.read_csv('split/' + file))
# dfs[24]

D:\apps\python64b3.9\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (14,15,16,17,18,23,24,25,26,27,28,29,30,31) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
D:\apps\python64b3.9\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
D:\apps\python64b3.9\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (16,17,18,19,20,21,22,23,24,25,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [117]:
# optional
instances = pd.concat([df[['Participant', 'Stimulus']].drop_duplicates() for df in dfs], ignore_index=True)
# instances

In [118]:
# optional
chopped = []
for df in dfs:
    grouped = df.groupby(['Participant', 'Stimulus'])
    for name, group in grouped:
        chopped.append(group[['Participant', 'Stimulus']])
        # print(group[['Participant', 'Stimulus']])

In [119]:
# optional
len(chopped)

2909

Participant data

In [120]:
participants = pd.read_csv('Metadata_Participants.csv')

classes

In [121]:
class experiment:
    def __init__(self, df, parti, stimu, asd):
        self.data = df
        self.participant = parti
        self.stimulus = stimu
        self.asd = (asd == 'ASD')
        
        # extract other frame-independent info
        self.trackRatio = self.data['Tracking Ratio [%]'][0]

    def preprocess(self):

        # delete columns that are information outputs
        self.data = self.data.drop(index=0)
        self.data = self.data.reset_index(drop=True)
        
        # drop the unused columns
        fs = list(self.data.columns)
        for f in fs:
            if f not in features: self.data.drop(columns=[f], inplace=True)

        # some data are missing Category Left. Copy Category Right over if that's the case.
        if 'Category Left' not in self.data.columns:
            self.data['Category Left'] = self.data['Category Right']
    
        # one hot encoding
        result = pd.DataFrame()
        for category in categorical:
            self.data[[category]] = self.data[[category]].replace('-', 'Blink') # replace all unwanted categories with `Blink`
            self.data[[category]] = self.data[[category]].replace('Separator', 'Blink')
            self.data[[category]] = self.data[[category]].replace('Left Click', 'Blink')
            self.data[[category]] = self.data[[category]].replace('Right Click', 'Blink')
            
            encoder = OneHotEncoder(categories=[categorical[category]])
            encoder.fit(self.data[[category]])
            OHE = encoder.transform(self.data[[category]]).toarray()
            OHEDF = pd.DataFrame(OHE, columns=encoder.get_feature_names_out())
            
#             print(result.shape, OHEDF.shape)
            result = pd.concat([result, OHEDF], axis=1)
        
#         print(result.shape, OHEDF.shape)
        self.data = pd.concat([self.data, result], axis=1)

In [122]:
experiments = []

for df in dfs:

    # participant
    p = df['Participant'][0]
    if p == 'Unidentified(Neg)': p = -1
    elif p == 'Unidentified(Pos)': p = 0

    # stimulus
    s = df['Stimulus'][0]

    # class
    if p == -1: c = 'TD'
    elif p == 0: c = 'ASD'
    else: c = participants['Class'][p - 1]

    # call constructor
    experiments.append(experiment(df, p, s, c))

In [123]:
c = 0
for exp in experiments:
#     print(c)
    exp.preprocess()
    c += 1

(0, 0) (1593, 3)
(1593, 3) (1593, 3)
(1593, 6) (1593, 3)
(0, 0) (267, 3)
(267, 3) (267, 3)
(267, 6) (267, 3)
(0, 0) (1132, 3)
(1132, 3) (1132, 3)
(1132, 6) (1132, 3)
(0, 0) (822, 3)
(822, 3) (822, 3)
(822, 6) (822, 3)
(0, 0) (133, 3)
(133, 3) (133, 3)
(133, 6) (133, 3)
(0, 0) (146, 3)
(146, 3) (146, 3)
(146, 6) (146, 3)
(0, 0) (145, 3)
(145, 3) (145, 3)
(145, 6) (145, 3)
(0, 0) (133, 3)
(133, 3) (133, 3)
(133, 6) (133, 3)
(0, 0) (145, 3)
(145, 3) (145, 3)
(145, 6) (145, 3)
(0, 0) (145, 3)
(145, 3) (145, 3)
(145, 6) (145, 3)
(0, 0) (146, 3)
(146, 3) (146, 3)
(146, 6) (146, 3)
(0, 0) (121, 3)
(121, 3) (121, 3)
(121, 6) (121, 3)
(0, 0) (145, 3)
(145, 3) (145, 3)
(145, 6) (145, 3)
(0, 0) (146, 3)
(146, 3) (146, 3)
(146, 6) (146, 3)
(0, 0) (302, 3)
(302, 3) (302, 3)
(302, 6) (302, 3)
(0, 0) (146, 3)
(146, 3) (146, 3)
(146, 6) (146, 3)
(0, 0) (146, 3)
(146, 3) (146, 3)
(146, 6) (146, 3)
(0, 0) (146, 3)
(146, 3) (146, 3)
(146, 6) (146, 3)
(0, 0) (146, 3)
(146, 3) (146, 3)
(146, 6) (146, 3)
(0

(145, 3) (145, 3)
(145, 6) (145, 3)
(0, 0) (146, 3)
(146, 3) (146, 3)
(146, 6) (146, 3)
(0, 0) (146, 3)
(146, 3) (146, 3)
(146, 6) (146, 3)
(0, 0) (146, 3)
(146, 3) (146, 3)
(146, 6) (146, 3)
(0, 0) (146, 3)
(146, 3) (146, 3)
(146, 6) (146, 3)
(0, 0) (107, 3)
(107, 3) (107, 3)
(107, 6) (107, 3)
(0, 0) (301, 3)
(301, 3) (301, 3)
(301, 6) (301, 3)
(0, 0) (123, 3)
(123, 3) (123, 3)
(123, 6) (123, 3)
(0, 0) (129, 3)
(129, 3) (129, 3)
(129, 6) (129, 3)
(0, 0) (111, 3)
(111, 3) (111, 3)
(111, 6) (111, 3)
(0, 0) (121, 3)
(121, 3) (121, 3)
(121, 6) (121, 3)
(0, 0) (132, 3)
(132, 3) (132, 3)
(132, 6) (132, 3)
(0, 0) (125, 3)
(125, 3) (125, 3)
(125, 6) (125, 3)
(0, 0) (103, 3)
(103, 3) (103, 3)
(103, 6) (103, 3)
(0, 0) (146, 3)
(146, 3) (146, 3)
(146, 6) (146, 3)
(0, 0) (146, 3)
(146, 3) (146, 3)
(146, 6) (146, 3)
(0, 0) (104, 3)
(104, 3) (104, 3)
(104, 6) (104, 3)
(0, 0) (845, 3)
(845, 3) (845, 3)
(845, 6) (845, 3)
(0, 0) (109, 3)
(109, 3) (109, 3)
(109, 6) (109, 3)
(0, 0) (96, 3)
(96, 3) (96, 

(145, 3) (145, 3)
(145, 6) (145, 3)
(0, 0) (146, 3)
(146, 3) (146, 3)
(146, 6) (146, 3)
(0, 0) (120, 3)
(120, 3) (120, 3)
(120, 6) (120, 3)
(0, 0) (145, 3)
(145, 3) (145, 3)
(145, 6) (145, 3)
(0, 0) (146, 3)
(146, 3) (146, 3)
(146, 6) (146, 3)
(0, 0) (127, 3)
(127, 3) (127, 3)
(127, 6) (127, 3)
(0, 0) (146, 3)
(146, 3) (146, 3)
(146, 6) (146, 3)
(0, 0) (146, 3)
(146, 3) (146, 3)
(146, 6) (146, 3)
(0, 0) (2119, 3)
(2119, 3) (2119, 3)
(2119, 6) (2119, 3)
(0, 0) (377, 3)
(377, 3) (377, 3)
(377, 6) (377, 3)
(0, 0) (146, 3)
(146, 3) (146, 3)
(146, 6) (146, 3)
(0, 0) (145, 3)
(145, 3) (145, 3)
(145, 6) (145, 3)
(0, 0) (146, 3)
(146, 3) (146, 3)
(146, 6) (146, 3)
(0, 0) (146, 3)
(146, 3) (146, 3)
(146, 6) (146, 3)
(0, 0) (146, 3)
(146, 3) (146, 3)
(146, 6) (146, 3)
(0, 0) (144, 3)
(144, 3) (144, 3)
(144, 6) (144, 3)
(0, 0) (62, 3)
(62, 3) (62, 3)
(62, 6) (62, 3)
(0, 0) (146, 3)
(146, 3) (146, 3)
(146, 6) (146, 3)
(0, 0) (146, 3)
(146, 3) (146, 3)
(146, 6) (146, 3)
(0, 0) (146, 3)
(146, 3) (14

(172, 3) (172, 3)
(172, 6) (172, 3)
(0, 0) (173, 3)
(173, 3) (173, 3)
(173, 6) (173, 3)
(0, 0) (169, 3)
(169, 3) (169, 3)
(169, 6) (169, 3)
(0, 0) (170, 3)
(170, 3) (170, 3)
(170, 6) (170, 3)
(0, 0) (173, 3)
(173, 3) (173, 3)
(173, 6) (173, 3)
(0, 0) (173, 3)
(173, 3) (173, 3)
(173, 6) (173, 3)
(0, 0) (664, 3)
(664, 3) (664, 3)
(664, 6) (664, 3)
(0, 0) (171, 3)
(171, 3) (171, 3)
(171, 6) (171, 3)
(0, 0) (173, 3)
(173, 3) (173, 3)
(173, 6) (173, 3)
(0, 0) (174, 3)
(174, 3) (174, 3)
(174, 6) (174, 3)
(0, 0) (173, 3)
(173, 3) (173, 3)
(173, 6) (173, 3)
(0, 0) (174, 3)
(174, 3) (174, 3)
(174, 6) (174, 3)
(0, 0) (174, 3)
(174, 3) (174, 3)
(174, 6) (174, 3)
(0, 0) (173, 3)
(173, 3) (173, 3)
(173, 6) (173, 3)
(0, 0) (173, 3)
(173, 3) (173, 3)
(173, 6) (173, 3)
(0, 0) (174, 3)
(174, 3) (174, 3)
(174, 6) (174, 3)
(0, 0) (173, 3)
(173, 3) (173, 3)
(173, 6) (173, 3)
(0, 0) (249, 3)
(249, 3) (249, 3)
(249, 6) (249, 3)
(0, 0) (174, 3)
(174, 3) (174, 3)
(174, 6) (174, 3)
(0, 0) (173, 3)
(173, 3) (17

(174, 3) (174, 3)
(174, 6) (174, 3)
(0, 0) (174, 3)
(174, 3) (174, 3)
(174, 6) (174, 3)
(0, 0) (173, 3)
(173, 3) (173, 3)
(173, 6) (173, 3)
(0, 0) (174, 3)
(174, 3) (174, 3)
(174, 6) (174, 3)
(0, 0) (174, 3)
(174, 3) (174, 3)
(174, 6) (174, 3)
(0, 0) (171, 3)
(171, 3) (171, 3)
(171, 6) (171, 3)
(0, 0) (174, 3)
(174, 3) (174, 3)
(174, 6) (174, 3)
(0, 0) (173, 3)
(173, 3) (173, 3)
(173, 6) (173, 3)
(0, 0) (174, 3)
(174, 3) (174, 3)
(174, 6) (174, 3)
(0, 0) (174, 3)
(174, 3) (174, 3)
(174, 6) (174, 3)
(0, 0) (302, 3)
(302, 3) (302, 3)
(302, 6) (302, 3)
(0, 0) (173, 3)
(173, 3) (173, 3)
(173, 6) (173, 3)
(0, 0) (174, 3)
(174, 3) (174, 3)
(174, 6) (174, 3)
(0, 0) (174, 3)
(174, 3) (174, 3)
(174, 6) (174, 3)
(0, 0) (173, 3)
(173, 3) (173, 3)
(173, 6) (173, 3)
(0, 0) (173, 3)
(173, 3) (173, 3)
(173, 6) (173, 3)
(0, 0) (167, 3)
(167, 3) (167, 3)
(167, 6) (167, 3)
(0, 0) (173, 3)
(173, 3) (173, 3)
(173, 6) (173, 3)
(0, 0) (173, 3)
(173, 3) (173, 3)
(173, 6) (173, 3)
(0, 0) (170, 3)
(170, 3) (17

...

(984, 3) (984, 3)
(984, 6) (984, 3)
(0, 0) (354, 3)
(354, 3) (354, 3)
(354, 6) (354, 3)
(0, 0) (711, 3)
(711, 3) (711, 3)
(711, 6) (711, 3)
(0, 0) (339, 3)
(339, 3) (339, 3)
(339, 6) (339, 3)
(0, 0) (711, 3)
(711, 3) (711, 3)
(711, 6) (711, 3)
(0, 0) (1002, 3)
(1002, 3) (1002, 3)
(1002, 6) (1002, 3)
(0, 0) (1458, 3)
(1458, 3) (1458, 3)
(1458, 6) (1458, 3)
(0, 0) (1070, 3)
(1070, 3) (1070, 3)
(1070, 6) (1070, 3)
(0, 0) (357, 3)
(357, 3) (357, 3)
(357, 6) (357, 3)
(0, 0) (949, 3)
(949, 3) (949, 3)
(949, 6) (949, 3)
(0, 0) (904, 3)
(904, 3) (904, 3)
(904, 6) (904, 3)
(0, 0) (883, 3)
(883, 3) (883, 3)
(883, 6) (883, 3)
(0, 0) (1057, 3)
(1057, 3) (1057, 3)
(1057, 6) (1057, 3)
(0, 0) (986, 3)
(986, 3) (986, 3)
(986, 6) (986, 3)
(0, 0) (358, 3)
(358, 3) (358, 3)
(358, 6) (358, 3)
(0, 0) (710, 3)
(710, 3) (710, 3)
(710, 6) (710, 3)
(0, 0) (341, 3)
(341, 3) (341, 3)
(341, 6) (341, 3)
(0, 0) (1973, 3)
(1973, 3) (1973, 3)
(1973, 6) (1973, 3)
(0, 0) (707, 3)
(707, 3) (707, 3)
(707, 6) (707, 3)
(0,

(0, 0) (25193, 3)
(25193, 3) (25193, 3)
(25193, 6) (25193, 3)
(0, 0) (27763, 3)
(27763, 3) (27763, 3)
(27763, 6) (27763, 3)
(0, 0) (8926, 3)
(8926, 3) (8926, 3)
(8926, 6) (8926, 3)
(0, 0) (23862, 3)
(23862, 3) (23862, 3)
(23862, 6) (23862, 3)
(0, 0) (23041, 3)
(23041, 3) (23041, 3)
(23041, 6) (23041, 3)
(0, 0) (22794, 3)
(22794, 3) (22794, 3)
(22794, 6) (22794, 3)
(0, 0) (26347, 3)
(26347, 3) (26347, 3)
(26347, 6) (26347, 3)
(0, 0) (66, 3)
(66, 3) (66, 3)
(66, 6) (66, 3)
(0, 0) (66, 3)
(66, 3) (66, 3)
(66, 6) (66, 3)
(0, 0) (5876, 3)
(5876, 3) (5876, 3)
(5876, 6) (5876, 3)
(0, 0) (66, 3)
(66, 3) (66, 3)
(66, 6) (66, 3)
(0, 0) (103, 3)
(103, 3) (103, 3)
(103, 6) (103, 3)
(0, 0) (76, 3)
(76, 3) (76, 3)
(76, 6) (76, 3)
(0, 0) (66, 3)
(66, 3) (66, 3)
(66, 6) (66, 3)
(0, 0) (61, 3)
(61, 3) (61, 3)
(61, 6) (61, 3)
(0, 0) (56, 3)
(56, 3) (56, 3)
(56, 6) (56, 3)
(0, 0) (100, 3)
(100, 3) (100, 3)
(100, 6) (100, 3)
(0, 0) (247, 3)
(247, 3) (247, 3)
(247, 6) (247, 3)
(0, 0) (104, 3)
(104, 3) (104

(0, 0) (145, 3)
(145, 3) (145, 3)
(145, 6) (145, 3)
(0, 0) (146, 3)
(146, 3) (146, 3)
(146, 6) (146, 3)
(0, 0) (2368, 3)
(2368, 3) (2368, 3)
(2368, 6) (2368, 3)
(0, 0) (146, 3)
(146, 3) (146, 3)
(146, 6) (146, 3)
(0, 0) (145, 3)
(145, 3) (145, 3)
(145, 6) (145, 3)
(0, 0) (146, 3)
(146, 3) (146, 3)
(146, 6) (146, 3)
(0, 0) (145, 3)
(145, 3) (145, 3)
(145, 6) (145, 3)
(0, 0) (145, 3)
(145, 3) (145, 3)
(145, 6) (145, 3)
(0, 0) (145, 3)
(145, 3) (145, 3)
(145, 6) (145, 3)
(0, 0) (146, 3)
(146, 3) (146, 3)
(146, 6) (146, 3)
(0, 0) (146, 3)
(146, 3) (146, 3)
(146, 6) (146, 3)
(0, 0) (146, 3)
(146, 3) (146, 3)
(146, 6) (146, 3)
(0, 0) (145, 3)
(145, 3) (145, 3)
(145, 6) (145, 3)
(0, 0) (1738, 3)
(1738, 3) (1738, 3)
(1738, 6) (1738, 3)
(0, 0) (2285, 3)
(2285, 3) (2285, 3)
(2285, 6) (2285, 3)
(0, 0) (140, 3)
(140, 3) (140, 3)
(140, 6) (140, 3)
(0, 0) (126, 3)
(126, 3) (126, 3)
(126, 6) (126, 3)
(0, 0) (132, 3)
(132, 3) (132, 3)
(132, 6) (132, 3)
(0, 0) (112, 3)
(112, 3) (112, 3)
(112, 6) (112, 

(106, 3) (106, 3)
(106, 6) (106, 3)
(0, 0) (146, 3)
(146, 3) (146, 3)
(146, 6) (146, 3)
(0, 0) (116, 3)
(116, 3) (116, 3)
(116, 6) (116, 3)
(0, 0) (118, 3)
(118, 3) (118, 3)
(118, 6) (118, 3)
(0, 0) (146, 3)
(146, 3) (146, 3)
(146, 6) (146, 3)
(0, 0) (146, 3)
(146, 3) (146, 3)
(146, 6) (146, 3)
(0, 0) (145, 3)
(145, 3) (145, 3)
(145, 6) (145, 3)
(0, 0) (1940, 3)
(1940, 3) (1940, 3)
(1940, 6) (1940, 3)
(0, 0) (145, 3)
(145, 3) (145, 3)
(145, 6) (145, 3)
(0, 0) (146, 3)
(146, 3) (146, 3)
(146, 6) (146, 3)
(0, 0) (126, 3)
(126, 3) (126, 3)
(126, 6) (126, 3)
(0, 0) (65, 3)
(65, 3) (65, 3)
(65, 6) (65, 3)
(0, 0) (129, 3)
(129, 3) (129, 3)
(129, 6) (129, 3)
(0, 0) (145, 3)
(145, 3) (145, 3)
(145, 6) (145, 3)
(0, 0) (146, 3)
(146, 3) (146, 3)
(146, 6) (146, 3)
(0, 0) (146, 3)
(146, 3) (146, 3)
(146, 6) (146, 3)
(0, 0) (146, 3)
(146, 3) (146, 3)
(146, 6) (146, 3)
(0, 0) (145, 3)
(145, 3) (145, 3)
(145, 6) (145, 3)
(0, 0) (22086, 3)
(22086, 3) (22086, 3)
(22086, 6) (22086, 3)
(0, 0) (145, 3)
(1

(0, 0) (91, 3)
(91, 3) (91, 3)
(91, 6) (91, 3)
(0, 0) (144, 3)
(144, 3) (144, 3)
(144, 6) (144, 3)
(0, 0) (99, 3)
(99, 3) (99, 3)
(99, 6) (99, 3)
(0, 0) (64, 3)
(64, 3) (64, 3)
(64, 6) (64, 3)
(0, 0) (66, 3)
(66, 3) (66, 3)
(66, 6) (66, 3)
(0, 0) (61, 3)
(61, 3) (61, 3)
(61, 6) (61, 3)
(0, 0) (263, 3)
(263, 3) (263, 3)
(263, 6) (263, 3)
(0, 0) (88, 3)
(88, 3) (88, 3)
(88, 6) (88, 3)
(0, 0) (120, 3)
(120, 3) (120, 3)
(120, 6) (120, 3)
(0, 0) (132, 3)
(132, 3) (132, 3)
(132, 6) (132, 3)
(0, 0) (146, 3)
(146, 3) (146, 3)
(146, 6) (146, 3)
(0, 0) (146, 3)
(146, 3) (146, 3)
(146, 6) (146, 3)
(0, 0) (146, 3)
(146, 3) (146, 3)
(146, 6) (146, 3)
(0, 0) (145, 3)
(145, 3) (145, 3)
(145, 6) (145, 3)
(0, 0) (146, 3)
(146, 3) (146, 3)
(146, 6) (146, 3)
(0, 0) (146, 3)
(146, 3) (146, 3)
(146, 6) (146, 3)
(0, 0) (145, 3)
(145, 3) (145, 3)
(145, 6) (145, 3)


In [132]:
experiments[0].data.loc[1, :]

Category Right                   Blink
Category Left                    Blink
Point of Regard Right X [px]    0.0000
Point of Regard Right Y [px]    0.0000
Point of Regard Left X [px]     0.0000
Point of Regard Left Y [px]     0.0000
Category Right_Blink               1.0
Category Right_Saccade             0.0
Category Right_Fixation            0.0
Category Left_Blink                1.0
Category Left_Saccade              0.0
Category Left_Fixation             0.0
Name: 1, dtype: object

In [136]:
[list(experiments[0].data.loc[1, :]) for i in range(exp.data.shape[0])]

[['Blink',
  'Blink',
  '0.0000',
  '0.0000',
  '0.0000',
  '0.0000',
  1.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0],
 ['Blink',
  'Blink',
  '0.0000',
  '0.0000',
  '0.0000',
  '0.0000',
  1.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0],
 ['Blink',
  'Blink',
  '0.0000',
  '0.0000',
  '0.0000',
  '0.0000',
  1.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0],
 ['Blink',
  'Blink',
  '0.0000',
  '0.0000',
  '0.0000',
  '0.0000',
  1.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0],
 ['Blink',
  'Blink',
  '0.0000',
  '0.0000',
  '0.0000',
  '0.0000',
  1.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0],
 ['Blink',
  'Blink',
  '0.0000',
  '0.0000',
  '0.0000',
  '0.0000',
  1.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0],
 ['Blink',
  'Blink',
  '0.0000',
  '0.0000',
  '0.0000',
  '0.0000',
  1.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0],
 ['Blink',
  'Blink',
  '0.0000',
  '0.0000',
  '0.0000',
  '0.0000',
  1.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0],
 ['Blink',
  'Blink',
  '0.0000',
  '0.0000',
  '0.0000',
  '0.0000',
  1.0,
  0.0,
  0.0,
  1.0

In [ ]:
# [TODO] copied from gpt

# Example time series data
# data = np.array([[i+j for j in range(5)] for i in range(100)])
# labels = np.array([i for i in range(100)])

# actual time series data
data = np.array([[exp.data[i].toarray() for i in exp.data.shape[0]] for exp in experiments])
labels = np.array([1 if exp.asd else 0 for exp in experiments])

# Reshape data to (samples, timesteps, features)
data = data.reshape((data.shape[0], data.shape[1], 1))

# Normalize data
scaler = MinMaxScaler()
data = scaler.fit_transform(data.reshape(-1, data.shape[-1])).reshape(data.shape)

# Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)


In [ ]:
# [TODO] copied from gpt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

model.summary()

In [ ]:
# [TODO] copied from gpt

history = model.fit(X_train, y_train, epochs=200, batch_size=32, validation_data=(X_test, y_test), verbose=1)

In [ ]:
# [TODO] copied from gpt

loss = model.evaluate(X_test, y_test, verbose=0)
print(f'Test Loss: {loss}')

In [ ]:
# [TODO] copied from gpt

predictions = model.predict(X_test)
print(predictions[:5])

In [7]:
master = pd.concat([df[['Participant', 'Stimulus']].drop_duplicates() for df in dfs], ignore_index=True)

In [8]:
master

,Participant,Stimulus
0,50,20 eye tracking (ballon droite).avi
1,52,20 eye tracking (ballon droite).avi
2,49,20 eye tracking (ballon droite).avi
3,55,20 eye tracking (ballon droite).avi
4,53,20 eye tracking (ballon droite).avi
...,...,...
2904,39,FEDE Drte.avi
2905,37,FEDE Drte.avi
2906,30,FEDE Drte.avi
2907,23,FEDE Drte.avi


In [9]:
master = master.applymap(str).drop_duplicates()

In [10]:
master

,Participant,Stimulus
0,50,20 eye tracking (ballon droite).avi
1,52,20 eye tracking (ballon droite).avi
2,49,20 eye tracking (ballon droite).avi
3,55,20 eye tracking (ballon droite).avi
4,53,20 eye tracking (ballon droite).avi
...,...,...
2903,39,FEDE Drte.avi
2905,37,FEDE Drte.avi
2906,30,FEDE Drte.avi
2907,23,FEDE Drte.avi


In [26]:
master = master.applymap(str).drop_duplicates()

master

,Participant,Trial
0,50,Trial020
1,52,Trial020
2,49,Trial020
3,55,Trial020
4,53,Trial020
...,...,...
2912,24,Trial001
2919,25,Trial019
2920,26,Trial019
3231,Unidentified(Pos),Trial010


In [27]:
master['Participant'].unique()

array(['50', '52', '49', '55', '53', '46', '42', 'Unidentified(Neg)',
       'Unidentified(Pos)', '20', '19', '27', '17', '15', '21', '23',
       '28', '18', '43', '48', '51', '44', '45', '47', '13', '14', '54',
       '59', '25', '26', '58', '29', '56', '30', '57', '31', '36', '2',
       '3', '4', '5', '6', '1', '9', '10', '8', '34', '40', '38', '32',
       '35', '11', '33', '39', '37', '7', '41', '22', '24'], dtype=object)